Simple Vanilla Neural Net 

In [94]:
#Image classifer with logistic regression
#images shape(width,height,3(rgb))

###to knit to html ipython nbconvert --to html [NEWFILENAME].html
import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
import math
from scipy import ndimage

#from basicFunctions import crossEntropyLoss 
 

In [70]:
def crossEntropyLoss(a,Y):
    num_of_examples = Y.shape[1] #len(Y) doesn't work, need 2nd dimesnion
    a = a.T
    loss = - (1.0 / num_of_examples) * (np.dot((1 - Y), np.log(1 - a)) + np.dot(Y, np.log(a)))
    return loss

In [71]:
# Testing cross entropy function
test_y = np.asarray([[1,1,1]])
test_x = np.asarray([[.8,.9,.4]])
print(crossEntropyLoss(test_x,test_y))


[[ 0.4149316]]


Expected output = [1231. 12312. 1231. ]

In [24]:
def sigmoid(x):
    output = 1.0/(1.0+np.exp(-x))
    cached_value = x
    cached_value = {"out_a":x}
    return output, cached_value

In [25]:
def relu(x,leaky=0):
    output = np.maximum(leaky,x)
#     cached_value = x
    cached_value = {"out_a":x}
    return output, cached_value
    

In [26]:
# To generate numbers randomly with numpy, we will use np.random.randn(dimension_x, dimension_y)

In [36]:
def initialize_params_shallow(input_layer, hidden_layer, output_layer):
    w1 = np.random.randn(hidden_layer,input_layer)
    w2 = np.random.randn(output_layer,hidden_layer)
    #np.random.randn gives numbers larger than 1 so: (vanishing/exploading gradients)
    w1,w2 = w1 * .01, w2 * .01
    
    #is it by one because of broadcasting 
    b1 = np.zeros((hidden_layer,1))
    b2 = np.zeros((output_layer,1))
    
    assert(w1.shape == (hidden_layer, input_layer))
    assert(w2.shape == (output_layer, hidden_layer))
    assert(b1.shape == (hidden_layer,1))
    assert(b2.shape == (output_layer,1))
    
    return {"w1":w1,"w2":w2,"b1":b1,"b2":b2}

In [41]:
np.random.seed(1)
params_shallow = initialize_params_shallow(3, 4, 2)
print("w1:", params_shallow["w1"])
print("w2:", params_shallow["w2"])
print("b1:", params_shallow["b1"])
print("b2:", params_shallow["b2"])

w1: [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]
 [ 0.01744812 -0.00761207  0.00319039]
 [-0.0024937   0.01462108 -0.02060141]]
w2: [[-0.00322417 -0.00384054  0.01133769 -0.01099891]
 [-0.00172428 -0.00877858  0.00042214  0.00582815]]
b1: [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
b2: [[ 0.]
 [ 0.]]


#### test case for initialize_params_shallow using seed(1)
##### (3,4,2)
<table style:"width-80%">
    <tr>
        <td>
         <b> w1:</b> [[ 0.01624345 -0.00611756 -0.00528172]
         [-0.01072969  0.00865408 -0.02301539]
         [ 0.01744812 -0.00761207  0.00319039]
         [-0.0024937   0.01462108 -0.02060141]]
        </td>
        <td>
        w2: [[-0.00322417 -0.00384054  0.01133769 -0.01099891]
 [-0.00172428 -0.00877858  0.00042214  0.00582815]]
        </td>
    </tr>
    <tr>
    <td>
        b1: [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
        </td>
        <td>
b2: [[ 0.]
 [ 0.]]
 </td>
     </tr>
</table>

In [54]:
def initialize_params_deep(input_layer, hidden_layers, output_layer):
#     np.random.seed(3) #non random random numbers (will get the same random numbers each time) used for testing
    layers = {} #dictionary of weights and biasis
    previous_layer = input_layer #sets the begining shape as shape of input layer
    for i in range(len(hidden_layers)):
        cur_layer = hidden_layers[i]
        weights = np.random.randn(cur_layer,previous_layer)
        weights *= .01 #because np.random.rand returns numbers too large, more efficeint durring gradient descensed 
        bias = np.zeros((cur_layer,1))
        cur_weights_key = "w" + str(i + 1) # i + 1 because humans like things indexed at 0
        cur_bias_key = "b" + str(i + 1)
        layers[cur_weights_key] = weights
        layers[cur_bias_key] = bias 
        
        assert(layers["w" + str(i + 1)].shape == (cur_layer, previous_layer))
        assert(layers["b" + str(i + 1)].shape == (cur_layer,1))
        
        previous_layer = cur_layer

    layers["w" + str(len(hidden_layers) + 1)] = np.random.randn(output_layer,previous_layer) * .01
    layers["b" + str(len(hidden_layers) + 1)] = np.zeros((output_layer,1))

    return layers

In [30]:
params_deep1 = initialize_params_deep(5,[4],3)
for param in params_deep2:
    print(param, params_deep2[param].shape)


NameError: name 'params_deep2' is not defined

In [40]:
np.random.seed(3)
params_deep2 = initialize_params_deep(5,[4],3)
for param in params_deep2:
    print(param, params_deep2[param].shape)
    print(params_deep2[param])


w1 (4, 5)
[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 (4, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
w2 (3, 4)
[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 (3, 1)
[[ 0.]
 [ 0.]
 [ 0.]]


#### Testcase initialize_params_deep 
##### seed(3) (5,[3],3)
<table style:"width-80%">
    <tr>
        <td>
w1 (4, 5)
[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
 </td>
 <td>
b1 (4, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
</td>
<tr>
<td>
w2 (3, 4)
[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
</td>
<td>
b2 (3, 1)
[[ 0.]
 [ 0.]
 [ 0.]]
 </td>
 </tr>
 </table>

In [66]:
def linear_forward_prop(a,w,b): #a because a is from the past layer
    w = w.T
    a = a.T
    z = np.dot(a,w) + b
#     print("z",z.shape,"a",a.shape[0],"w",w.shape[1],"b",b.shape)
    assert(z.shape == (a.shape[0], w.shape[1] ))
    #     assert(z.shape == np.dot(a,w).shape)
    cached_inputs = {"in_a":a,"w":w,"b":b} #(a,w,b)
    return z,cached_inputs
    

In [42]:
np.random.seed(1)
X = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1) #(1,1) broadcasting

X = np.array(X)
w = np.array(w)
b = np.array(b)
z,_ = linear_forward_prop(X,w,b)
print("X",X.shape,"w",w.shape,"b",b.shape,"z",z.shape)
print(z)

X (3, 2) w (1, 3) b (1,) z (2, 1)
[[ 3.26295337]
 [-1.23429987]]


##### Test case for linear_foward_prop
##### seed(1) x=(3,2) w=(1,3) b=(1)
<table style:"width-80%">
    <tr>
        <td>
X (3, 2) w (1, 3) b (1,) z (2, 1)
</td>
</tr>
    <tr>
        <td>
[[ 3.26295337]
 [-1.23429987]]
 </td>
</tr>
</table>

In [58]:
def singleFowardPropagation(X,w,b,activationType,leaky=0): #TODO change x to a
#     cached_inputs = {"w":None,"x":None,"b":None,"z":None}
    z,cached_values = linear_forward_prop(X,w,b)
    if activationType == "sigmoid":
        a, activation_cache = sigmoid(z)
    if activationType == "relu":
        a, activation_cache = relu(z,leaky=leaky)
#     final_cache = c
    cached_values.update(activation_cache)
    return a, cached_values  #add asserts 

In [59]:
np.random.seed(2)
X = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1) #(1,1) broadcasting
a,cached_values = singleFowardPropagation(X,w,b,"relu")
print("Relu A",a)

a,cached_values = singleFowardPropagation(X,w,b,"sigmoid")
print("Sigmoud A",a)
# print(X,w,b)

Relu A [[ 3.43896131]
 [ 0.        ]]
Sigmoud A [[ 0.96890023]
 [ 0.11013289]]


Relu A [[ 3.43896131]
 [ 0.        ]]
Sigmoud A [[ 0.96890023]
 [ 0.11013289]]

In [60]:
def fowardPropagate(X,params):
    print("len of params", len(params))
    num_layers = len(params)/2
    #assert len params is even
    num_layers = int(num_layers)
    a = X
    cached_values = []
    for i in range(1,num_layers):
        w = params["w" + str(i)]
        b = params["b" + str(i)]
        past_a = a
        a, cur_cached = singleFowardPropagation(past_a,w,b,"relu")
        cached_values += [cur_cached] #TODO: get foramt of cached values 
    w = params["w" + len(num_layers)]
    b = params["b" + len(num_layers)]
    a, cur_cached = singleFowardProgagation(a,w,b,"sigmoid")
    cached_values += [cur_cached] 
    #write assert statemt here
    #get shape of a at the end 
    
    return a, cached_values

In [67]:
np.random.seed(6)
X = np.random.randn(5,4)
w1 = np.random.randn(4,5)
b1 = np.random.randn(4,1)
w2 = np.random.randn(3,4)
b2 = np.random.randn(3,1)
w3 = np.random.randn(1,3)
b3 = np.random.randn(1,1)
params = {
    "w1":w1,
    "b1":b1,
    "w2":w2,
    "b2":b2,
    "w3":w3,
    "b3":b3
}
print(params)
fowardPropagate(X, params)


{'w1': array([[ 0.35480861,  1.81259031, -1.3564758 , -0.46363197,  0.82465384],
       [-1.17643148,  1.56448966,  0.71270509, -0.1810066 ,  0.53419953],
       [-0.58661296, -1.48185327,  0.85724762,  0.94309899,  0.11444143],
       [-0.02195668, -2.12714455, -0.83440747, -0.46550831,  0.23371059]]), 'b1': array([[ 1.38503523],
       [-0.51962709],
       [-0.78015214],
       [ 0.95560959]]), 'w2': array([[-0.12673638, -1.36861282,  1.21848065, -0.85750144],
       [-0.56147088, -1.0335199 ,  0.35877096,  1.07368134],
       [-0.37550472,  0.39636757, -0.47144628,  2.33660781]]), 'b2': array([[ 1.50278553],
       [-0.59545972],
       [ 0.52834106]]), 'w3': array([[ 0.9398248 ,  0.42628539, -0.75815703]]), 'b3': array([[-0.16236698]])}
len of params 6


ValueError: operands could not be broadcast together with shapes (4,3) (3,1) 

In [81]:
def linearBackprop(cached_values, dZ):
    a = cached_values["in_a"]
    w = cached_values["w"]
    b = cached_values["b"]
    number_examples = a.shape[1]
    dW = (1/number_examples) * np.dot(dZ,a.T)
    dB = (1/number_examples) * np.sum(dZ, axis = 1, keepdims= True)
    dA = np.dot(w.T, dZ)
#write asserts for dW dB & dA TODO
    return {"dW":dW,"dB":dB,"dA":dA}
    
    

In [82]:
np.random.seed(1)
dZ = np.random.randn(1,2)
a = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1)
cached_values = {"w": w, "in_a":a,"b":b}
linearBackprop(cached_values, dZ)

{'dA': array([[ 0.51822968, -0.19517421],
        [-0.40506361,  0.15255393],
        [ 2.37496825, -0.89445391]]),
 'dB': array([[ 0.50629448]]),
 'dW': array([[-0.10076895,  1.40685096,  1.64992505]])}

{'dA': array([[ 0.51822968, -0.19517421],
        [-0.40506361,  0.15255393],
        [ 2.37496825, -0.89445391]]),
 'dB': array([[ 0.50629448]]),
 'dW': array([[-0.10076895,  1.40685096,  1.64992505]])}

In [92]:
def relu_backward(dA, cache):
    z = cache
    dZ = np.array(dA, copy=True) #convert dA to np array
    dZ[z < 0] = 0 
    
    return dZ
    

In [84]:
def sigmoid_backward(dA, cache):
    z = cache
    s = 1/(1 + np.exp(-z))
    dz = dA * s * (1-s)
    
    return dz

In [87]:
def backwards_activation(dA, cached_values, activation_type):
    if activation_type == "sigmoid":
        dZ = sigmoid_backward(dA, cached_values["z"])
    if activation_type == "relu":
        dZ = relu_backward(dA, cached_values["z"])
    gradients = linearBackprop(cached_values, dZ)
    
    return gradients
    
    
        
        
    

In [93]:
np.random.seed(2)
dA = np.random.randn(1,2)
a = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1)
z = np.random.randn(1,2)
cached_values = {"w": w, "in_a":a,"b":b, "z":z}
backwards_activation(dA, cached_values, activation_type="relu")
#working


{'dA': array([[ 0.44090989,  0.        ],
        [ 0.37883606,  0.        ],
        [-0.2298228 ,  0.        ]]),
 'dB': array([[-0.20837892]]),
 'dW': array([[ 0.44513824,  0.37371418, -0.10478989]])}